In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas.plotting as pd_plt
import matplotlib.colors as plt_colors
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, precision_score, recall_score, confusion_matrix, classification_report
from IPython.display import display_html
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import category_encoders as ce
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, validation_curve, train_test_split, KFold,GridSearchCV, StratifiedKFold, RepeatedStratifiedKFold, TunedThresholdClassifierCV, cross_validate
from sklearn.svm import SVC

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedKFold
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.svm import SVR
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder, TargetEncoder
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer, KBinsDiscretizer
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor

In [7]:
path_train = 'train.csv'

In [9]:
df = pd.read_csv(path_train)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40455 non-null  float64
 1   cut      40455 non-null  object 
 2   color    40455 non-null  object 
 3   clarity  40455 non-null  object 
 4   depth    40455 non-null  float64
 5   table    40455 non-null  float64
 6   price    40455 non-null  int64  
 7   x        40455 non-null  float64
 8   y        40455 non-null  float64
 9   z        40455 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.1+ MB


In [11]:
df = df.drop_duplicates()
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.55,Ideal,I,SI1,61.7,55.0,1134,5.29,5.31,3.27
1,0.94,Premium,E,SI1,62.4,56.0,4580,6.27,6.24,3.90
2,0.33,Premium,E,SI2,60.0,59.0,594,4.49,4.47,2.69
3,0.31,Ideal,D,VS2,59.4,56.0,879,4.45,4.40,2.63
4,2.01,Very Good,H,SI1,62.8,59.0,17759,7.99,8.04,5.03
...,...,...,...,...,...,...,...,...,...,...
40450,1.50,Very Good,D,SI1,60.7,62.0,11442,7.30,7.33,4.44
40451,1.51,Ideal,D,SI1,61.9,57.0,11834,7.35,7.42,4.57
40452,0.42,Ideal,G,SI2,60.9,56.0,971,4.87,4.89,2.97
40453,2.05,Premium,E,SI2,59.7,59.0,17237,8.39,8.27,4.97


In [13]:
df['y']=np.where(df['y']>=15, df['y']/10,df['y'])
df['z']=np.where(df['z']>=15, df['z']/10,df['z'])

In [15]:
axis_features = ['x','y','z']
num_features = ['carat','depth','table']

y = np.array(df.price)
X = df.drop(columns=['price'])

X_train,X_test,y_train,y_test = train_test_split(X, y) 

In [17]:
axis_transformer = Pipeline(steps=[
    ('missing_num', IterativeImputer(missing_values=0, max_iter=20)),
    ('polynom', PolynomialFeatures(3,include_bias=False)),
    ('scaler', StandardScaler())])
num = Pipeline(steps=[
    ('missing_num', KNNImputer()),
    ('bin', KBinsDiscretizer(n_bins = 3, strategy = 'uniform', encode = 'onehot-dense')),
    ('scaler', StandardScaler())])
clarity_map = [{
    'col':'clarity',##### Обратить внимание
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}
    }]
clarity_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', MinMaxScaler())
    ])    
color_map = [
  {'col': 'color',
   'mapping': {'D': 6, 'E': 5, 'F': 4, 'G': 3, 'H': 2, 'I': 1, 'J': 0}}
]
color_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=color_map)),
    ('scaler', MinMaxScaler())
    ])    
cut_map = [
  {'col': 'cut',
   'mapping': {'Ideal': 4, 'Premium': 3, 'Very Good': 2, 'Good': 1, 'Fair': 0}}
]
cut_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=cut_map)),
    ('scaler', MinMaxScaler())
    ])   

In [19]:
CT = ColumnTransformer([
        ("axis_transformer", axis_transformer, axis_features),
        ("num", num, num_features),# убрать быстую разряженную 
        ("ordinal_map", clarity_transformer, ['clarity']),
        ("color_map", color_transformer, ['color']),
        ("cut_map", cut_transformer, ['cut'])
        ]).set_output(transform='pandas')

display(CT)

ct = CT.fit_transform(X_train)
pd.DataFrame(ct).head().T

ColumnTransformer(transformers=[('axis_transformer',
                                 Pipeline(steps=[('missing_num',
                                                  IterativeImputer(max_iter=20,
                                                                   missing_values=0)),
                                                 ('polynom',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['x', 'y', 'z']),
                                ('num',
                                 Pipeline(steps=[('missing_num', KNNImputer()),
                                                 ('bin',
                                                  KBinsDiscretizer(encode='onehot-dense',
                                                                   n_bins=3,
                                                                   strateg...
                                                  OrdinalEncoder(mapping=[{'col': 'color',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': D    6
E    5
F    4
G    3
H    2
I    1
J    0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['color']),
                                ('cut_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'cut',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': Ideal        4
Premium      3
Very Good    2
Good         1
Fair         0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['cut'])])

,28012,32936,9184,36403,20814
axis_transformer__x,-0.978547,1.156668,-1.139358,-1.184028,1.594432
axis_transformer__y,-0.951544,1.091438,-1.212541,-1.212541,1.568433
axis_transformer__z,-1.034619,0.902349,-1.121349,-1.309264,1.335999
axis_transformer__x^2,-0.940744,1.138908,-1.062818,-1.095869,1.670684
axis_transformer__x y,-0.930994,1.101742,-1.090865,-1.107294,1.655441
axis_transformer__x z,-0.964523,0.993497,-1.058455,-1.144553,1.513895
axis_transformer__y^2,-0.920548,1.063579,-1.118135,-1.118135,1.638557
axis_transformer__y z,-0.955076,0.957234,-1.086689,-1.156039,1.498712
axis_transformer__z^2,-0.983158,0.845831,-1.048627,-1.185646,1.351515
axis_transformer__x^3,-0.887512,1.093851,-0.976918,-1.000508,1.717236


In [42]:
model = Pipeline(steps=[
  ('preproc', CT),
  ('svc', SVR(kernel='rbf'))
])
#cv = RepeatedKFold(n_splits=5, n_repeats= 10)

C= np.array([ 150000, 200000]) #np.linspace(20, 25, 10) ,1,10**4, 10**5,10**6,10**7
gamma= np.array([1]) #linspace(0.07, 0.15, 10)[0.01,0.1,1,5,10,100],1,5,10,100

param_grid =[
    {'svc__C': C,
     'svc__gamma': gamma,
     }
]

grid = GridSearchCV(model, param_grid, return_train_score=True, scoring='neg_mean_squared_error', n_jobs=-1) #cv=cv, 
grid.fit(X_train, y_train)


KeyboardInterrupt: 

In [34]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score'])
display(grid_result)
pv_grid_result_mean = grid_result.pivot_table(index='param_svc__C',columns='param_svc__gamma',values='mean_test_score')
pv_grid_result_std = grid_result.pivot_table(index='param_svc__C',columns='param_svc__gamma',values='std_test_score')

f, (ax1, ax2) = plt.subplots(1,2,figsize=(15, 5))
sns.heatmap(pv_grid_result_mean, annot=True, linewidths=.5, ax=ax1, fmt=".2f")
sns.heatmap(pv_grid_result_std, annot=True, linewidths=.5, ax=ax2);

,14,13,11,10,12,9,7,8,6,3,4,5,0,1,2
mean_fit_time,356.878293,121.206551,100.993371,83.127427,88.987536,82.568264,79.336497,79.319123,82.451972,84.554149,83.646228,84.573171,109.807044,98.941145,85.041425
std_fit_time,59.643985,2.921069,2.011887,0.425092,2.817532,0.565604,0.398918,0.737625,3.911783,0.492345,0.929475,0.493522,0.393039,4.150312,0.733989
mean_score_time,13.627265,24.407807,24.194543,24.310005,24.182654,24.185301,24.100934,24.933803,24.222123,24.585611,24.658855,24.273544,24.068541,24.40198,24.38904
std_score_time,1.743922,0.258274,0.416505,0.086191,0.492186,0.102455,0.050231,0.20929,0.224276,0.470263,0.143837,0.073935,0.120443,0.707849,0.482095
param_svc__C,10000.0,10000.0,1000.0,1000.0,10000.0,1000.0,100.0,100.0,100.0,10.0,10.0,10.0,0.01,0.01,0.01
param_svc__gamma,1.0,0.1,1.0,0.1,0.01,0.01,0.1,1.0,0.01,0.01,0.1,1.0,0.01,0.1,1.0
params,"{'svc__C': 10000.0, 'svc__gamma': 1.0}","{'svc__C': 10000.0, 'svc__gamma': 0.1}","{'svc__C': 1000.0, 'svc__gamma': 1.0}","{'svc__C': 1000.0, 'svc__gamma': 0.1}","{'svc__C': 10000.0, 'svc__gamma': 0.01}","{'svc__C': 1000.0, 'svc__gamma': 0.01}","{'svc__C': 100.0, 'svc__gamma': 0.1}","{'svc__C': 100.0, 'svc__gamma': 1.0}","{'svc__C': 100.0, 'svc__gamma': 0.01}","{'svc__C': 10.0, 'svc__gamma': 0.01}","{'svc__C': 10.0, 'svc__gamma': 0.1}","{'svc__C': 10.0, 'svc__gamma': 1.0}","{'svc__C': 0.01, 'svc__gamma': 0.01}","{'svc__C': 0.01, 'svc__gamma': 0.1}","{'svc__C': 0.01, 'svc__gamma': 1.0}"
split0_test_score,-473569.32183,-454397.820581,-671973.581293,-630863.607432,-597152.496228,-781236.993707,-1173510.989319,-1601067.749895,-1401576.375031,-2275553.190394,-3138239.604594,-5859272.788071,-17683741.004245,-17680329.163996,-17847293.130092
split1_test_score,-441520.277907,-478219.639837,-603549.93363,-642881.810019,-634894.515929,-843874.882246,-1066219.793391,-1410503.329488,-1474672.193727,-2314377.891387,-2814999.860586,-5919202.419555,-18201600.872109,-18209577.435124,-18378232.51483
split2_test_score,-452846.172634,-455005.19275,-565829.856531,-660196.004684,-683027.82853,-916433.529064,-1088245.41424,-1430790.202265,-1595704.919729,-2445226.906946,-2988652.269662,-6150057.743002,-18703377.202027,-18701883.345255,-18873028.118339


KeyError: 'mean_test_score'